In [ ]:
import pandas as pd
import scipy.stats as stats
from EFF3D.MRI import open_npy
import torch
import torch.nn as nn
import tqdm
import numpy as np
from fastai.vision.all import *
from EFF3D.res import resnet10

In [ ]:
df=pd.read_csv('../nonorm/ADNCold.csv')
# path=Path('/home/staff/xin/Downloads/newMRI/ADtrain')

In [ ]:
df.head()

,name,label
0,/home/staff/xin/Downloads/newMRI/ADtrain/AD/157.nii.gz,AD
1,/home/staff/xin/Downloads/newMRI/ADtrain/AD/382.nii.gz,AD
2,/home/staff/xin/Downloads/newMRI/ADtrain/AD/309.nii.gz,AD
3,/home/staff/xin/Downloads/newMRI/ADtrain/AD/014_S_4615.nii.gz,AD
4,/home/staff/xin/Downloads/newMRI/ADtrain/AD/285.nii.gz,AD


In [ ]:
temp1=open_npy(df.iat[0,0])
temp2=open_npy(df.iat[1,0])

In [ ]:
encoder=resnet10(shortcut_type='B')
encoder=nn.DataParallel(encoder)
encoder=nn.Sequential(*list(encoder.module.children()))
head=nn.Sequential(nn.AdaptiveMaxPool3d(1), Flatten(), nn.Linear(512, 2))
model=nn.Sequential(encoder,head)
model.load_state_dict(torch.load('./models/medres10max.pth'))

<All keys matched successfully>

In [ ]:
model[0][:5]

Sequential(
  (0): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
  (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
)

In [ ]:
# proj = nn.Conv3d(1,1, kernel_size=25, stride=25)
proj=model[0][:5]

In [ ]:
result=None
for name in tqdm.tqdm(df[df.label=='AD'].name):
    temp=open_npy(name)
    with torch.no_grad():
        re=proj(temp.unsqueeze(0))
    if result==None:
        result=re[0]
    else:
        result=torch.cat((result,re[0]))
        

    

100%|██████████| 478/478 [10:50<00:00,  1.36s/it]


In [ ]:
result2=None
for name in tqdm.tqdm(df[df.label=='NC'].name):
    temp=open_npy(name)
    with torch.no_grad():
        re=proj(temp.unsqueeze(0))
    if result2==None:
        result2=re[0]
    else:
        result2=torch.cat((result2,re[0]))

  9%|▉         | 55/596 [00:42<06:58,  1.29it/s]


KeyboardInterrupt: 

In [ ]:
result.shape

torch.Size([30592, 46, 55, 46])

In [ ]:
p=[]
for i in range(392):
    p.append(stats.ttest_ind(result.flatten(1)[:,i],result2.flatten(1)[:478,i])[1])

In [ ]:
p=np.asarray(p)

In [ ]:
sort=np.argsort(p)

In [ ]:
p.shape

(392,)

In [ ]:
p[sort<60]

array([1.00000000e+00, 6.22274504e-03, 1.00000000e+00, 1.57079925e-01,
       2.37987523e-01, 4.97634674e-01, 6.54055246e-01, 1.12507421e-01,
       1.00000000e+00, 2.86824761e-04, 3.06876952e-03, 9.11736111e-01,
       2.03090754e-02, 4.30938044e-01, 2.26946894e-01, 2.61225460e-05,
       2.23839943e-06, 1.27668512e-03, 9.95024069e-01, 5.07022796e-01,
       4.59688952e-02, 4.66376865e-01, 8.43902893e-01, 1.00000000e+00,
       1.22606488e-01, 1.00000000e+00, 7.85558335e-04, 1.40421005e-03,
       5.92569504e-01, 5.99972592e-02, 6.45509286e-01, 7.67711514e-02,
       1.00000000e+00, 8.50220083e-01, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.95577615e-01, 1.48650707e-01,
       9.22443337e-01, 3.61899575e-01, 1.40897967e-01, 1.00000000e+00,
       1.00000000e+00, 1.49845638e-01, 6.19826304e-02, 1.46206717e-01,
       3.43834011e-01, 1.46937040e-02, 2.74382506e-01, 1.00000000e+00,
      

In [ ]:
np.save('sort.npz',sort)

In [ ]:
temp.shape

torch.Size([1, 181, 217, 181])

In [ ]:
p[360]


0.004668511904601649